## Context

This dataset was created by Yaroslav Bulatov by taking some publicly available fonts and extracting glyphs from them to make a dataset similar to MNIST. There are 10 classes, with letters A-J.

## Content

A set of training and test images of letters from A to J on various typefaces. The images size is 28x28 pixels.

## Acknowledgements

The dataset can be found on Tensorflow github page as well as on the blog from Yaroslav, here.

## Inspiration

This is a pretty good dataset to train classifiers! According to Yaroslav:

Judging by the examples, one would expect this to be a harder task than MNIST. This seems to be the case -- logistic regression on top of stacked auto-encoder with fine-tuning gets about 89% accuracy whereas same approach gives got 98% on MNIST. Dataset consists of small hand-cleaned part, about 19k instances, and large uncleaned dataset, 500k instances. Two parts have approximately 0.5% and 6.5% label error rate. I got this by looking through glyphs and counting how often my guess of the letter didn't match it's unicode value in the font file.
Enjoy!

In [1]:
import numpy as np
import tensorflow as tf
import os
import pandas as pd


/Users/ryanleung/anaconda3/envs/py3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [2]:
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from matplotlib import pyplot as plt


In [3]:
from sklearn.model_selection import train_test_split

In [4]:
from sklearn.preprocessing import OneHotEncoder

## Getting the path names to images

In [5]:
parentDir = 'notMNIST_small/'
print(parentDir)
data = []
total = 0
good = 0
for folder in os.listdir(parentDir):
    if folder != '.DS_Store':
        for file in os.listdir(parentDir + folder):
            if total % 1000 == 0:
                print(total, good)
            total += 1
            try:
                img_path = parentDir + folder + '/' + file
                img = Image.open(img_path)
                data.append([img_path, folder])
                good += 1
            except:
                pass
            
dataset = pd.DataFrame(data)
dataset.head()


                                

notMNIST_small/
0 0
1000 1000
2000 2000
3000 3000
4000 3999
5000 4999
6000 5999
7000 6998
8000 7998
9000 8998
10000 9998
11000 10998
12000 11998
13000 12998
14000 13998
15000 14998
16000 15998
17000 16998
18000 17998


,0,1
0,notMNIST_small/I/Qml0d2lzZS50dGY=.png,I
1,notMNIST_small/I/RW5nbGFuZCBCb2xkSXRhbGljLnR0Z...,I
2,notMNIST_small/I/R3JlZWsgSXRhbGljLnR0Zg==.png,I
3,notMNIST_small/I/Rmx5d2hlZWxTcXVhcmUudHRm.png,I
4,notMNIST_small/I/SGFuZGljYXAub3Rm.png,I


In [6]:
print(len(dataset))

18724


## Set batch size and epochs

Don't want batch size to be too large or not too small

In [13]:
batch_size = 16
num_epochs = 100
def input_func(features, labels, batch_size, mode):
    
    def parser(image, label): 
        
        img = tf.image.decode_png(tf.read_file(image))
        img = tf.reshape(img, [28, 28, 1])
        img = tf.cast(img, tf.float32, "cast")

        return img, label
    
    dataset = tf.data.Dataset.from_tensor_slices((features, labels))
    dataset = dataset.map(parser)
    if mode == tf.estimator.ModeKeys.TRAIN: 
        dataset = dataset.repeat(12)
    dataset = dataset.batch(batch_size)
    
    return dataset
    

## Define model architecture
Uses a two layer, each layer consisting of a convolutional and pooling layer, architecture. (Same architecture as original MNIST CNN)

In [64]:
def my_model(features, labels, mode, params):
    #initialize input by reshaping and casting for network
    #img = tf.image.decode_png(tf.read_file(features['x'][0]))
    # img = np.array( img, dtype='uint8' ).flatten()
    
    # FIRST LAYER
    # ---conv layer with 32 filters, 5x5 kernel, and relu activation
    # ---pool layer with 2x2 pool window and stride of 2x2
    conv1 = tf.layers.conv2d(inputs=features, filters=64, kernel_size=(5, 5), padding="same", activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=(2, 2), strides=(2, 2))
    
    # SECOND LAYER
    # ---conv layer with 64 filters, 5x5 kernel, and relu activation
    # ---pool layer with 2x2 pool window and stride of 2x2
    conv2 = tf.layers.conv2d(inputs=pool1, filters=128, kernel_size=(5, 5), padding="same", activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=(2, 2), strides=(2, 2))
    
#     print(pool2.shape)
    
    # DENSE LAYER
    # ---flatten output into vector
    # ---dropout to prevent overfitting
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 128])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(inputs=dense, rate=0.2, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    logits = tf.layers.dense(inputs=dropout, units=10)
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    onehot_labels = tf.reshape(onehot_labels, [-1, 10])
    loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)
#     loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(loss=loss,global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    print(labels.shape)
    print(predictions["classes"].shape)
    eval_metric_ops = {"accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)



## Training and Evaluating
-> split data into train and test (2:1)
-> instantiate model with my_mode as cnn
-> convert dataset (np array) to dataframe to use pd.factorize to get integer labels, then convert back to np array

In [12]:
# Fetch the data
X_train, X_test, y_train, y_test = train_test_split(dataset[0], pd.factorize(dataset[1])[0], test_size=0.20, random_state=42)

# Build CNN.
classifier = tf.estimator.Estimator(model_fn=my_model)

# Train the Model.
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

classifier.train(input_fn=lambda:input_func(X_train, y_train, batch_size, tf.estimator.ModeKeys.TRAIN), steps = 10000)

# Evaluate the model.

eval_result = classifier.evaluate(input_fn=lambda:input_func(X_test, y_test, batch_size, tf.estimator.ModeKeys.EVAL))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_tf_random_seed': None, '_keep_checkpoint_every_n_hours': 10000, '_global_id_in_cluster': 0, '_task_type': 'worker', '_num_worker_replicas': 1, '_save_summary_steps': 100, '_master': '', '_task_id': 0, '_keep_checkpoint_max': 5, '_num_ps_replicas': 0, '_log_step_count_steps': 100, '_service': None, '_save_checkpoints_secs': 600, '_train_distribute': None, '_save_checkpoints_steps': None, '_session_config': None, '_is_chief': True, '_model_dir': '/var/folders/th/svpqqvhs62790bm9gczzcth40000gn/T/tmpgzpz4h9z', '_evaluation_master': '', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a27491c50>}
INFO:tensorflow:Calling model_fn.
(?, 7, 7, 128)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into 

INFO:tensorflow:global_step/sec: 9.85202
INFO:tensorflow:loss = 0.016113032, step = 6801 (10.150 sec)
INFO:tensorflow:global_step/sec: 10.1423
INFO:tensorflow:loss = 0.048931718, step = 6901 (9.860 sec)
INFO:tensorflow:global_step/sec: 9.8569
INFO:tensorflow:loss = 0.0025207207, step = 7001 (10.146 sec)
INFO:tensorflow:global_step/sec: 9.9018
INFO:tensorflow:loss = 0.0018015531, step = 7101 (10.099 sec)
INFO:tensorflow:global_step/sec: 10.2388
INFO:tensorflow:loss = 0.0020498815, step = 7201 (9.767 sec)
INFO:tensorflow:global_step/sec: 10.275
INFO:tensorflow:loss = 0.009991124, step = 7301 (9.732 sec)
INFO:tensorflow:global_step/sec: 9.50265
INFO:tensorflow:loss = 0.03612329, step = 7401 (10.523 sec)
INFO:tensorflow:global_step/sec: 8.46723
INFO:tensorflow:loss = 0.0052235397, step = 7501 (11.811 sec)
INFO:tensorflow:global_step/sec: 9.51629
INFO:tensorflow:loss = 0.0025830683, step = 7601 (10.507 sec)
INFO:tensorflow:global_step/sec: 10.3355
INFO:tensorflow:loss = 0.15688726, step = 7

In [51]:
predict_sample = {
    'a': [2, 89, 3671],
    'b': [23, 37, 39],
    'c': [81],
    'd': [47, 3647],
    'e': [95, 3648],
    'f': [3709], 
    'g': [3718, 3704],
    'h': [22, 35, 40],
    'i': [3715, 3691],
    'j': [3725, 3732]
}

In [69]:
class_dict = {}
i = 0
while len(class_dict.keys()) < 10:
    class_dict[y_test[i]] = X_test[i].split('/')[1]
    i += 1
print(class_dict)

{0: 'I', 1: 'G', 2: 'A', 3: 'F', 4: 'H', 5: 'J', 6: 'C', 7: 'D', 8: 'E', 9: 'B'}


In [70]:
tf.logging.set_verbosity(tf.logging.ERROR)
template = ('\nPrediction is "{}" ({:.3f}%)')
for k, v in predict_sample.items():
    for i in v:
        predictions = classifier.predict(input_fn=lambda:input_func([X_test[i]], [y_test[i]], 1, tf.estimator.ModeKeys.PREDICT))
        for pred_dict in predictions:
            pred_dict = pred_dict
        pred_class = pred_dict['classes']
        pred_certainty = float(pred_dict['probabilities'][pred_class] * 100) / float(sum(pred_dict['probabilities']))
        
        display(Image(filename=X_test[i], width=100, height=100))
        print(template.format(class_dict[pred_class], pred_certainty, class_dict[y_test[i]]))

(?, 7, 7, 128)



Prediction is "I" (99.989%)
(?, 7, 7, 128)



Prediction is "I" (99.999%)
(?, 7, 7, 128)



Prediction is "F" (100.000%)
(?, 7, 7, 128)



Prediction is "J" (99.999%)
(?, 7, 7, 128)


...


Prediction is "H" (99.965%)
(?, 7, 7, 128)



Prediction is "D" (100.000%)
(?, 7, 7, 128)



Prediction is "D" (90.928%)
(?, 7, 7, 128)



Prediction is "E" (99.990%)
(?, 7, 7, 128)



Prediction is "E" (88.257%)
